In [253]:
pd.set_option('display.max_colwidth', None)

In [286]:
import pandas as pd
import time
import numpy as np
data = pd.read_csv("news.csv",index_col="Unnamed: 0").sample(frac=1)
data.fillna("",inplace=True)
data["Title"]=[" ".join(a) for a in tuple(zip(data["Site"],data["Tags"],data["Authors"],data["Title"]))]
yesData = data[data["Verdict"]==1]
noData = data[data["Verdict"]==0].iloc[0:len(yesData)]
data=pd.concat([yesData,noData],ignore_index=True).sample(frac=1)

In [287]:
trainData, valData, testData = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

C:\Users\james\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [242]:
import sklearn
from sklearn.linear_model import SGDClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier

In [288]:
classifierDict = {"hinge":{},"log_loss":{},"huber":{}}
for loss in ['hinge', 'log_loss', 'huber']:
        for alpha in [alpha/10000 for alpha in range(1,400,20)]:
            text_clf = Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss=loss,penalty="l2",alpha=alpha,n_iter_no_change=200)),
            ])
            
            text_clf.fit(trainData["Title"], trainData["Verdict"])
            predicted = text_clf.predict(valData["Title"])
            valData["Predicted"]=predicted
            captured = len(valData[(valData["Predicted"]==1)&(valData["Verdict"]==1)])
            totalPredicted = len(valData[valData["Predicted"]==1])
            totalVerdict = len(valData[valData["Verdict"]==1])
            ct = pd.crosstab(valData["Predicted"],valData["Verdict"],normalize="columns")
            print(loss,alpha,ct[1][0],ct[0][1])
            classifierDict[loss][alpha]=text_clf

hinge 0.0001 0.22018348623853212 0.21782178217821782
hinge 0.0021 0.21100917431192662 0.19801980198019803
hinge 0.0041 0.22935779816513763 0.22772277227722773
hinge 0.0061 0.24770642201834864 0.2079207920792079
hinge 0.0081 0.21100917431192662 0.21782178217821782
hinge 0.0101 0.25688073394495414 0.21782178217821782
hinge 0.0121 0.12844036697247707 0.27722772277227725
hinge 0.0141 0.12844036697247707 0.24752475247524752
hinge 0.0161 0.05504587155963303 0.38613861386138615
hinge 0.0181 0.045871559633027525 0.40594059405940597
hinge 0.0201 0.12844036697247707 0.2871287128712871
hinge 0.0221 0.07339449541284404 0.32673267326732675
hinge 0.0241 0.05504587155963303 0.36633663366336633
hinge 0.0261 0.027522935779816515 0.45544554455445546
hinge 0.0281 0.045871559633027525 0.4158415841584158
hinge 0.0301 0.07339449541284404 0.3564356435643564
hinge 0.0321 0.12844036697247707 0.2871287128712871
hinge 0.0341 0.13761467889908258 0.24752475247524752
hinge 0.0361 0.11926605504587157 0.3168316831683

In [289]:
testData["Predicted"]=classifierDict["hinge"][0.0221].predict(testData["Title"])
c = classifierDict["hinge"][0.0221]

In [290]:
pd.crosstab(testData["Verdict"],testData["Predicted"])

Predicted,0,1
Verdict,,
0,58,45
1,19,88


In [292]:
testData[(testData["Verdict"]==1)&(testData["Predicted"]==0)]

,Site,Hash,Tags,Time,Authors,Title,Verdict,Link,Predicted
211,ABC,5904430:a276aa,"regional communities,children,child health and behaviour,family and relationships,parenting,psychology",12/02/2024 4:39,Sophie Holder,"ABC regional communities,children,child health and behaviour,family and relationships,parenting,psychology Sophie Holder The importance of teaching children financial empathy amid a cost of living crisis",1,https://www.abc.net.au/news/2024-02-12/teach-children-cost-of-living-crisis-financial-empathy-parents/103438718,0
137,SMH,6282414:0c8b91,"national,nsw",13/02/2024 10:00,,"SMH national,nsw Mr Minns goes to Canberra",1,https://www.smh.com.au/national/nsw/mr-minns-goes-to-canberra-20240213-p5f4gs.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
281,SMH,6282414:37da42,"national,nsw",14/02/2024 5:49,Angus Dalton,"SMH national,nsw Angus Dalton Major train delays hit Sydney, services skipping several stations",1,https://www.smh.com.au/national/nsw/major-train-delays-hit-sydney-services-skipping-several-stations-20240214-p5f4zr.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
296,ABC,5904430:693caf,"courts and trials,media industry",13/02/2024 1:10,Elizabeth Byrne and Victor Petrovic,"ABC courts and trials,media industry Elizabeth Byrne and Victor Petrovic YouTuber to face contempt proceedings after broadcasting Bruce Lehrmann defamation hearing",1,https://www.abc.net.au/news/2024-02-13/youtuber-glenn-logan-to-face-contempt-proceedings-bruce-lehrmann/103458956,0
86,SMH,6282414:de0898,national,6/02/2024 18:00,Millie Muroi,SMH national Millie Muroi Everything I’ve learnt from Ross Gittins (the rock star),1,https://www.smh.com.au/national/everything-i-ve-learnt-from-ross-gittins-the-rock-star-20240123-p5ezgk.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
136,SMH,6282414:d25bb3,"national,queensland",11/02/2024 7:49,Felicity Caldwell,"SMH national,queensland Felicity Caldwell Did you go to a public or private school? Your subject choices may reveal all",1,https://www.smh.com.au/national/queensland/did-you-go-to-a-public-or-private-school-your-subject-choices-may-reveal-all-20240207-p5f33d.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
491,SMH,6282414:ccb173,"business,the economy",13/02/2024 18:00,Ross Gittins,"SMH business,the economy Ross Gittins Want better productivity? Start by ensuring our kids can read",1,https://www.smh.com.au/business/the-economy/want-better-productivity-start-by-ensuring-our-kids-can-read-20240213-p5f4j8.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
181,SMH,6282414:590e2b,"national,nsw",8/02/2024 6:19,Jordan Baker,"SMH national,nsw Jordan Baker Perrottet breaks silence to call for negative gearing review",1,https://www.smh.com.au/national/nsw/perrottet-breaks-silence-to-call-for-negative-gearing-review-20240208-p5f3hb.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
470,SMH,6282414:f0e6b6,"business,markets",13/02/2024 0:57,Stephen Bartholomeusz,"SMH business,markets Stephen Bartholomeusz The danger sign that has the world on edge",1,https://www.smh.com.au/business/markets/the-danger-sign-that-has-the-world-on-edge-20240213-p5f4fo.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0
146,SMH,6282414:3df7be,"national,nsw",9/02/2024 5:00,Bevan Shields,"SMH national,nsw Bevan Shields The Festival of Ross Gittins isn’t over yet",1,https://www.smh.com.au/national/nsw/the-festival-of-ross-gittins-isn-t-over-yet-20240209-p5f3mg.html?ref=rss&utm_medium=rss&utm_source=rss_feed,0


In [293]:
import _pickle as cPickle
# save the classifier
with open('classifier.pkl', 'wb') as fid:
    cPickle.dump(c, fid)    

# load it again
#with open('classifier.pkl', 'rb') as fid:
#    gnb_loaded = cPickle.load(fid)